In [1]:
import pandas as pd
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import pyspark
from pyspark.sql.functions import split
from pyspark.sql.functions import explode, col, trim, rtrim, ltrim
from pyspark.sql.functions import regexp_replace, regexp_extract
from pyspark.sql.functions import lower
from pyspark.sql.functions import desc
from pyspark.ml.feature import StopWordsRemover
import re
import glob
import os
import shutil
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from googletrans import Translator
translator = Translator()
from langdetect import detect

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cedua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_twitter = pd.read_csv('C:\\\\Users\\\\cedua\\\\CDIA - PUCSP\\\\PROJETO PySpark - SAVINO\\\\Contador-de-palavras---PySpark\\\\collecting_data\\\\twitter_whey_V3.csv')
df_twitter.head()

,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,kamii,@kami_whey,2022-06-15T22:28:20.000Z,"kamii\n@kami_whey\n·\nJun 15, 2022","i will not give context, i just find this imag...",NaN,1,NaN,3,['https://pbs.twimg.com/media/FVU7ZmGWAAIqepU?...,https://twitter.com/kami_whey/status/153720035...
1,tia feri.,@feristhii,2022-06-15T21:29:34.000Z,"tia feri.\n@feristhii\n·\nJun 15, 2022",Apenas um treino usando uma gostosa,NaN,3,58,869,['https://pbs.twimg.com/media/FVUuIaeWUAA13vp?...,https://twitter.com/feristhii/status/153718557...
2,Monty Riches,@MontyRiches,2022-06-15T22:34:27.000Z,"Monty Riches\n@MontyRiches\n·\nJun 15, 2022","MONTY'S APE WATCH Mega ""UNDER 10 $SOL"" Editi...",🚨 🦧 🚨 💪 🦍,4,3,19,['https://pbs.twimg.com/media/FVU8-5cXsAEsOqc?...,https://twitter.com/MontyRiches/status/1537201...
3,Walk Like an Egyptian Bot,@egyptian_bot,2022-06-15T22:10:49.000Z,Walk Like an Egyptian Bot\n@egyptian_bot\n·\nJ...,Foreign types with the hookah pipes say ay oh ...,NaN,NaN,NaN,NaN,[],https://twitter.com/egyptian_bot/status/153719...
4,thomas the gain,@thomasphall,2022-06-15T20:49:42.000Z,"thomas the gain\n@thomasphall\n·\nJun 15, 2022",I use to have my autocorrect change “way” to “...,🚂 😂,2,NaN,NaN,[],https://twitter.com/thomasphall/status/1537175...


In [66]:
#ONLY THE TEXT
df_twitter_text = df_twitter[['Embedded_text']]
df_twitter_text

,Embedded_text
0,"i will not give context, i just find this imag..."
1,Apenas um treino usando uma gostosa
2,"MONTY'S APE WATCH Mega ""UNDER 10 $SOL"" Editi..."
3,Foreign types with the hookah pipes say ay oh ...
4,I use to have my autocorrect change “way” to “...
...,...
8413,treino assim pra quem não sabe
8414,FELLOW GAYMERS RISE UP
8415,Shit is about to get incredibly annoying
8416,O TREINO INOVADOR DO CORINTHIANS


In [67]:
#CREATING A NEW FEATURE TO INDENTIFY THE LANGUAGE OF THE TEXT
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

df_twitter_text['lang'] = df_twitter_text['Embedded_text'].apply(detect_language)
df_twitter_text.head()

C:\Users\cedua\AppData\Local\Temp\ipykernel_10500\2608503998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_twitter_text['lang'] = df_twitter_text['Embedded_text'].apply(detect_language)


,Embedded_text,lang
0,"i will not give context, i just find this imag...",en
1,Apenas um treino usando uma gostosa,pt
2,"MONTY'S APE WATCH Mega ""UNDER 10 $SOL"" Editi...",en
3,Foreign types with the hookah pipes say ay oh ...,en
4,I use to have my autocorrect change “way” to “...,en


In [68]:
#HOW MANY LANGUAGES?
print(f"SHAPE -- > {df_twitter_text.shape}")
print(f"HOW MANY LANGUAGES -- > {len(set(df_twitter_text['lang']))}")
print(set(df_twitter_text['lang']))

SHAPE -- > (8418, 2)
HOW MANY LANGUAGES -- > 32
{'sq', 'fr', 'ja', 'tr', 'nl', 'th', 'de', 'ca', 'es', 'ar', 'en', 'da', 'hr', 'sw', 'ro', 'af', 'cs', 'sk', 'vi', 'lt', 'sl', 'so', 'no', 'tl', 'pt', 'hu', 'id', 'et', 'it', 'sv', 'fi', 'cy'}


In [69]:
#FILTERING ENGLISH AND PORTUGUESE
df_twitter_text = df_twitter_text.loc[df_twitter_text['lang'].isin(['en', 'pt'])]
df_twitter_text.shape
df_twitter_text.head()

,Embedded_text,lang
0,"i will not give context, i just find this imag...",en
1,Apenas um treino usando uma gostosa,pt
2,"MONTY'S APE WATCH Mega ""UNDER 10 $SOL"" Editi...",en
3,Foreign types with the hookah pipes say ay oh ...,en
4,I use to have my autocorrect change “way” to “...,en


In [70]:
#SEPARATING BETWEEN en AND pt TO TRANSLATE en
df_twitter_text_en = df_twitter_text[df_twitter_text['lang']=='en'].set_index('lang')
df_twitter_text_pt = df_twitter_text[df_twitter_text['lang']=='pt'].set_index('lang')

In [71]:
display(df_twitter_text_en)
display(df_twitter_text_pt)

,Embedded_text
lang,
en,"i will not give context, i just find this imag..."
en,"MONTY'S APE WATCH Mega ""UNDER 10 $SOL"" Editi..."
en,Foreign types with the hookah pipes say ay oh ...
en,I use to have my autocorrect change “way” to “...
en,smelling my brother's bottle of whey protein p...
...,...
en,who up soundin their fury rn
en,Working hard on my crossfit.\n#CatsOfTwitter #...
en,Change your life. \n\nAnyone can do it. Start ...


,Embedded_text
lang,
pt,Apenas um treino usando uma gostosa
pt,Tinha mais gente no treino que no jogo.
pt,saudade do crossfit dms
pt,Hoje eu não acabei o treino foi ele quem acabo...
pt,| Que fofura! \n\nJaqueline e Murilo levaram ...
...,...
pt,escuta kpop no treino
pt,#eleuthero\n#suplementos\n#sistemarespiratório...
pt,VEJA: Rodolffo (BBB 21) chama atenção por fal...


In [72]:
#SAVING TO CSV --> TXT --> WORD COUNT PYSPARK 
df_twitter_text_pt.to_csv('C:\\Users\\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data/twitter_pt_data.csv', index=False)
df_twitter_text_en.to_csv('C:\\Users\\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data/twitter_en_data.csv', index=False)

#### PySpark

In [2]:
def clean_twitter_data_PT(txt_path):
    #pyspark session
    spark = SparkSession.builder.appName("PySpark-LimpandoTwitterDataPT").getOrCreate()
    sc = spark.sparkContext

    #txt doc
    df = spark.read.text(f"{txt_path}")

    #spliting the df by space " " 
    df = df.select(
    split(df.value, ' ').alias('words'))

    #extracting words from the lists
    df = df.select(
    explode(col("words")).alias("words3"))
    
    #every word to lower case
    df = df.select(
    lower(
    col('words3')
    ).alias('words4'))

    #removing punctuation
    punc = r"""[|-:!?.,"'"\"\/]"""
    é_rem = r"\b(\w*é\w*)"
    https_rem = r"(http|https)://[^\s]*"

    df = df.withColumn("words5", regexp_replace("words4", punc, ""))
    df = df.withColumn("words77", regexp_replace("words5", https_rem, ""))
    df = df.withColumn("words7", regexp_replace("words77", é_rem, ""))
    
    # Removing the NULLs
    df = df.where(
        col("words7") != "")

    #FIRST COUNT#
    df_counted = df.groupBy(
        col("words7"),
    ).count()

    #RETAKING
    df2 = df.select(
        split(
            col("words7"),
            " "
        ).alias("word8"))
    
    #removing stopwords - portuguese
    stopwords_remover = StopWordsRemover(inputCol="word8", outputCol="words9",   
                                            stopWords=StopWordsRemover.loadDefaultStopWords("portuguese"))
    df2 = stopwords_remover.transform(df2)
    df2 = df2.drop("word8")

    #removing nulls - step2
    df2 = df2.select(
        (col("words9")[0]).alias("twitter_text_data_PT")) #LAST EDIT COL

    df2 = df2.where(
    col("twitter_text_data_PT") != "null")

    df2 = df2.where(
        col("twitter_text_data_PT") != "")
    
    #FINAL COUNT#
    df_counted2 = df2.groupby(
    col("twitter_text_data_PT")
    ).count()

    df_counted2 = df_counted2.orderBy(desc("count")).toPandas()
    
    return df_counted2

In [3]:
def clean_twitter_data_EN(txt_path):
    #pyspark session
    spark = SparkSession.builder.appName("PySpark-LimpandoTwitterDataEN").getOrCreate()
    sc = spark.sparkContext

    #txt doc
    df = spark.read.text(f"{txt_path}")

    #spliting the df by space " " 
    df = df.select(
    split(df['value'], ' ').alias('words'))
    
    #extracting words from the lists
    df = df.select(
    explode(col("words")).alias("words3"))
    
    #every word to lower case
    df = df.select(
    lower(
    col('words3')
    ).alias('words44'))

    #removing punctuation
    https_rem = r"(http|https)://[^\s]*"
    df = df.withColumn("words4", regexp_replace("words44", https_rem, ""))

    df = df.select(
    regexp_extract(
    col('words4'),  #cleaning words2 column
    '[A-z]+',       #selecting all the words
    0).alias('words5'))

    # Removing the NULLs
    df = df.where(
        col("words5") != "")
    
    #FIRST COUNT#
    df_counted = df.groupBy(
        col("words5"),
    ).count()

    #RETAKING
    df2 = df.select(
        split(
            col("words5"),
            " "
        ).alias("word6"))
    
    #removing stopwords - english 
    stopwords_remover2 = StopWordsRemover(inputCol="word6", outputCol="words7")
    df2 = stopwords_remover2.transform(df2)
    df2 = df2.drop("word6")

    #removing nulls - step2
    df2 = df2.select(
        (col("words7")[0]).alias("twitter_text_data_EN"))

    df2 = df2.where(
    col("twitter_text_data_EN") != "null")

    df2 = df2.where(
        col("twitter_text_data_EN") != "")
    
    #FINAL COUNT#
    df_counted2 = df2.groupby(
    col("twitter_text_data_EN")
    ).count()

    df_counted2 = df_counted2.orderBy(desc("count")).toPandas()
    
    return df_counted2

In [4]:
df_twitter_text_pt_pyspark = clean_twitter_data_PT("C:\\Users\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data\\twitter_pt_data.txt")
df_twitter_text_pt_pyspark.to_csv('C:\\Users\\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data\\twitter_pt_data_pyspark.csv',index=False)

df_twitter_text_en_pyspark = clean_twitter_data_EN("C:\\Users\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data\\twitter_en_data.txt")
df_twitter_text_en_pyspark.to_csv('C:\\Users\\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data\\twitter_en_data_pyspark.csv',index=False)

Py4JJavaError: An error occurred while calling o127.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-JO1VKGV executor driver): org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.PATTERN] The value of parameter(s) `regexp` in `regexp_replace` is invalid: '[|-:!?.,"\'"\\"\\/]'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidPatternError(QueryExecutionErrors.scala:2754)
	at org.apache.spark.sql.errors.QueryExecutionErrors.invalidPatternError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.generate_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo

In [81]:
# ddff = pd.read_csv('C:\\Users\\cedua\\CDIA - PUCSP\\PROJETO PySpark - SAVINO\\Contador-de-palavras---PySpark\\collecting_data/twitter_pt_data_pyspark.csv')
# ddff.head(20)

twitter data PT
- remover links
- remover | , \ , - , etc

twitter data EN 
- remover links